In [1]:
import pandas as pd

complexes_data = pd.read_csv('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/csv_files/complexes_details.csv')
complexes_data

,complex_id,complex_name
0,sr:complex:705,Nacional
1,sr:complex:1078,Estadio de la Cartuja
2,sr:complex:1495,Sibur Arena
3,sr:complex:2375,Complexo de Tenis do Jamor
4,sr:complex:4032,Shree Shiv Chhatrapati Sports Complex
...,...,...
652,sr:complex:82356,Dan Maskell Tennis Centre
653,sr:complex:82415,George P Mitchell 40 Tennis Center
654,sr:complex:82417,National Tennis Center
655,sr:complex:82467,Tcja Saint-Malo


save data in postgresql database

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


Creating DataBase:

In [3]:
pip install toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


Creating Complexes Table

In [5]:
complexes_create_table_sql = """create table if not exists complexes_table(
complex_id varchar(50) PRIMARY KEY, 
complex_name varchar(100) NOT NULL)"""

#execute:
mycursor.execute(complexes_create_table_sql)

print("Complex Table has been created")

Complex Table has been created


Fetching Column Name from complexes_table:

In [6]:
column_name = """SELECT column_name FROM 
        information_schema.columns WHERE table_name = 'complexes_table';"""
mycursor.execute(column_name)
fetching_column_names = mycursor.fetchall()
fetching_column_names

[('complex_id',), ('complex_name',)]

insert values into complexes table:

In [7]:
complex_insert_table_sql = """Insert into complexes_table(
complex_id, 
complex_name) 
values (%s, %s) 
ON CONFLICT (complex_id) DO NOTHING"""

#ON CONFLICT (cat_id) DO NOTHING
#“If there’s a conflict — like trying to insert a row with a duplicate primary key or unique value — then just skip that row and move on. 
# Don’t insert it, and don’t throw an error.”

In [8]:
for index, row in complexes_data.iterrows():
    print(row) #We will get the o/p in the format of dictionary with column names as keys.

    print("Inserting:", tuple(row))  #The iter row (only values) will converts into python tuple 
    print("----------------------\n")
    mycursor.execute(complex_insert_table_sql, tuple(row))
print("Data inserted into the complexes_table using iterrows()")


complex_id      sr:complex:705
complex_name          Nacional
Name: 0, dtype: object
Inserting: ('sr:complex:705', 'Nacional')
----------------------

complex_id            sr:complex:1078
complex_name    Estadio de la Cartuja
Name: 1, dtype: object
Inserting: ('sr:complex:1078', 'Estadio de la Cartuja')
----------------------

complex_id      sr:complex:1495
complex_name        Sibur Arena
Name: 2, dtype: object
Inserting: ('sr:complex:1495', 'Sibur Arena')
----------------------

complex_id                 sr:complex:2375
complex_name    Complexo de Tenis do Jamor
Name: 3, dtype: object
Inserting: ('sr:complex:2375', 'Complexo de Tenis do Jamor')
----------------------

complex_id                            sr:complex:4032
complex_name    Shree Shiv Chhatrapati Sports Complex
Name: 4, dtype: object
Inserting: ('sr:complex:4032', 'Shree Shiv Chhatrapati Sports Complex')
----------------------

complex_id      sr:complex:5520
complex_name           O2 Arena
Name: 5, dtype: object
Inser

Fetching Data from complexes_table:

In [9]:
#execute:
mycursor.execute("Select * from complexes_table")

fetching_Data = mycursor.fetchall()
fetching_Data

[('sr:complex:705', 'Nacional'),
 ('sr:complex:1078', 'Estadio de la Cartuja'),
 ('sr:complex:1495', 'Sibur Arena'),
 ('sr:complex:2375', 'Complexo de Tenis do Jamor'),
 ('sr:complex:4032', 'Shree Shiv Chhatrapati Sports Complex'),
 ('sr:complex:5520', 'O2 Arena'),
 ('sr:complex:5526', 'Ostravar Arena'),
 ('sr:complex:6310', 'Varazdin Arena'),
 ('sr:complex:6484', 'Arena Montpellier'),
 ('sr:complex:6736', 'Arenes de Metz'),
 ('sr:complex:6888', 'Palais des sports de Pau'),
 ('sr:complex:6890', 'Palais des Sports Orleans'),
 ('sr:complex:6904', 'Stade Pierre-de-Coubertin'),
 ('sr:complex:7048', 'Palacio de Deportes Jose Maria Martin Carpena'),
 ('sr:complex:7052', 'Pabellon Municipal Fuente San Luis'),
 ('sr:complex:7208', 'Unipol Arena'),
 ('sr:complex:7778', 'Arena Gripe'),
 ('sr:complex:9725', 'Arena Armeets Sofia'),
 ('sr:complex:10063', 'Kindarena'),
 ('sr:complex:11769', 'Estadio German Becker'),
 ('sr:complex:12241', 'Palais des sports de Gerland'),
 ('sr:complex:15014', 'Palais

close cursor & db connection:

In [10]:
mycursor.close()
my_db_connection.close()